In [1]:
# Plots'offline and notebook's style configs
from IPython.core.display import HTML
from sigi.apps.servicos.models import Servico
from django.db import connection
from django.core.urlresolvers import reverse
from django.db.models import Q, Count
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly 
import plotly.plotly as py
import plotly.graph_objs as go
import pandas as pd
import cufflinks as cf
import numpy as np
import datetime
import calendar

init_notebook_mode(connected=True)
css = open('style-table.css').read() + open('style-notebook.css').read()
HTML('<style>{}</style>'.format(css))

In [2]:
''' Function defined on sigi/sigi/sigi/apps/home/views.py line 322 to line 364
    All rights reserved.
    Obs.: Added number of deactivated types of services per month/year.
'''
def busca_informacoes_seit(mes_atual=None):
    if mes_atual is None:
        mes_atual = datetime.date.today().replace(day=1)
    mes_anterior = mes_atual - datetime.timedelta(days=1)
    proximo_mes = mes_atual + datetime.timedelta(days=calendar.monthrange(mes_atual.year, mes_atual.month)[1])
    
    meses = []
    mes = mes_atual
    for i in range(1, 13):
        meses.append(mes)
        mes = (mes - datetime.timedelta(days=1)).replace(day=1)
        
    result = {
        'mes_atual': mes_atual,
        'mes_anterior': mes_anterior,
        'proximo_mes': proximo_mes,
        'servicos': [],
    } 

    for tipo_servico in TipoServico.objects.all():
        result['servicos'].append(
            {'nome': tipo_servico.nome,
             'total_desativados': tipo_servico.servico_set.filter(data_desativacao__year=mes_atual.year, data_desativacao__month=mes_atual.month).count(),
             'total': tipo_servico.servico_set.filter(Q(data_ativacao__lt=proximo_mes)&(Q(data_desativacao=None)|Q(data_desativacao__gt=proximo_mes))).count(),
             'novos_mes_anterior': tipo_servico.servico_set.filter(data_ativacao__year=mes_anterior.year, data_ativacao__month=mes_anterior.month).count(),
             'novos_mes_atual': tipo_servico.servico_set.filter(data_ativacao__year=mes_atual.year, data_ativacao__month=mes_atual.month).count(),
             }
        )

    return result

In [3]:
# Define periods for data frame's indexes and variables
di = pd.date_range('2008-11', periods=((datetime.date.today().year - datetime.date(2008,11, 30).year) * 12) - (12 - datetime.date.today().month - 2), freq='M')
period = {}
period_formated = []
tabela_resumo_seit = []
services_names = []
services_names = []
period['year'] = di.year
period['month'] = di.month
dataframe_data = []
# Build of the data frame's data as a dictionary
for i in range(0, len(period['year'])):
    period_formated.append('{year}/{month}'.format(year = str(period['year'][i]), month = str(period['month'][i])))
    try:
        mes = datetime.date(year=period['year'][i], month=period['month'][i], day=1)
        tabela_resumo_seit.append(busca_informacoes_seit(mes))
    except:
        tabela_resumo_seit.append(busca_informacoes_seit())

In [4]:
# Build list of desired values into tabela_resumo_seit
ativados_temp = []
ativados = []
desativados_temp = []
desativados = []
ativos_temp = []
ativos = []

for i in range(0, len(period['year'])):
    for j in range (0,6):
        ativados_temp.append(tabela_resumo_seit[i]['servicos'][j]['novos_mes_atual'])
        desativados_temp.append(tabela_resumo_seit[i]['servicos'][j]['total_desativados'])
        ativos_temp.append(tabela_resumo_seit[i]['servicos'][j]['total'])

# Create an iterator
it = [iter(ativados_temp)] * 6  
# Unpack the copies of the iterator, and pass them as parameters to zip
ativados = zip(*it)
# Create an iterator
it2 = [iter(desativados_temp)] * 6  
# Unpack the copies of the iterator, and pass them as parameters to zip
desativados = zip(*it2)
# Create an iterator
it3 = [iter(ativos_temp)] * 6  
# Unpack the copies of the iterator, and pass them as parameters to zip
ativos = zip(*it3)
    
# Creation of the data frame
df_final = pd.DataFrame(ativados, index=period_formated, columns=['PortalModelo(Ativados)', 'SAPL(Ativados)', 'SAAP(Ativados)', 'SPDO(Ativados)', 'Dominio.Leg.br(Ativados)', 'Dominio.GOV.br(Ativados)'])
df_desativados = pd.DataFrame(desativados, index=period_formated, columns=['PortalModelo(Desativados)', 'SAPL(Desativados)', 'SAAP(Desativados)', 'SPDO(Desativados)', 'Dominio.Leg.br(Desativados)', 'Dominio.GOV.br(Desativados)'])
df_ativos = pd.DataFrame(ativos, index=period_formated, columns=['PortalModelo(Ativos)', 'SAPL(Ativos)', 'SAAP(Ativos)', 'SPDO(Ativos)', 'Dominio.Leg.br(Ativos)', 'Dominio.GOV.br(Ativos)'])

df_final['PortalModelo(Desativados)'] = df_desativados['PortalModelo(Desativados)']
df_final['SAPL(Desativados)'] = df_desativados['SAPL(Desativados)']
df_final['SAAP(Desativados)'] = df_desativados['SAAP(Desativados)']
df_final['SPDO(Desativados)'] = df_desativados['SPDO(Desativados)']
df_final['Dominio.Leg.br(Desativados)'] = df_desativados['Dominio.Leg.br(Desativados)']
df_final['Dominio.GOV.br(Desativados)'] = df_desativados['Dominio.GOV.br(Desativados)']

df_final['PortalModelo(Ativos)'] = df_ativos['PortalModelo(Ativos)']
df_final['SAPL(Ativos)'] = df_ativos['SAPL(Ativos)']
df_final['SAAP(Ativos)'] = df_ativos['SAAP(Ativos)']
df_final['SPDO(Ativos)'] = df_ativos['SPDO(Ativos)']
df_final['Dominio.Leg.br(Ativos)'] = df_ativos['Dominio.Leg.br(Ativos)']
df_final['Dominio.GOV.br(Ativos)'] = df_ativos['Dominio.GOV.br(Ativos)']

cols = df_final.columns.tolist()

cols.sort()

df_final = df_final[cols]

df_final


,Dominio.GOV.br(Ativados),Dominio.GOV.br(Ativos),Dominio.GOV.br(Desativados),Dominio.Leg.br(Ativados),Dominio.Leg.br(Ativos),Dominio.Leg.br(Desativados),PortalModelo(Ativados),PortalModelo(Ativos),PortalModelo(Desativados),SAAP(Ativados),SAAP(Ativos),SAAP(Desativados),SAPL(Ativados),SAPL(Ativos),SAPL(Desativados),SPDO(Ativados),SPDO(Ativos),SPDO(Desativados)
2008/11,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0
2008/12,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2009/1,0,0,0,0,0,0,2,3,0,0,0,0,0,0,0,0,0,0
2009/2,0,0,0,0,0,0,2,5,0,0,0,0,0,0,0,0,0,0
2009/3,0,0,0,0,0,0,6,11,0,0,0,0,0,0,0,0,0,0
2009/4,0,0,0,0,0,0,4,15,0,0,0,0,0,0,0,0,0,0
2009/5,0,0,0,0,0,0,8,23,0,0,0,0,0,0,0,0,0,0
2009/6,0,0,0,0,0,0,2,25,0,0,0,0,0,0,0,0,0,0
2009/7,0,0,0,0,0,0,6,31,0,0,0,0,0,0,0,0,0,0
2009/8,0,0,0,0,0,0,10,41,0,0,0,0,0,0,0,0,0,0


In [5]:
df_stats = pd.DataFrame(df_final.loc[:,:].sum(numeric_only=True), columns=['Total'])
df_stats['TipoServico'] = df_final.columns[:]

df_stats

,Total,TipoServico
Dominio.GOV.br(Ativados),160,Dominio.GOV.br(Ativados)
Dominio.GOV.br(Ativos),5957,Dominio.GOV.br(Ativos)
Dominio.GOV.br(Desativados),20,Dominio.GOV.br(Desativados)
Dominio.Leg.br(Ativados),1615,Dominio.Leg.br(Ativados)
Dominio.Leg.br(Ativos),38918,Dominio.Leg.br(Ativos)
Dominio.Leg.br(Desativados),5,Dominio.Leg.br(Desativados)
PortalModelo(Ativados),1547,PortalModelo(Ativados)
PortalModelo(Ativos),41070,PortalModelo(Ativos)
PortalModelo(Desativados),49,PortalModelo(Desativados)
SAAP(Ativados),59,SAAP(Ativados)


In [6]:
# Obs.: Layout dos plots sao normalmente configurados como
# dicionarios e encapsulados pela func. abaixo
layout = go.Layout(
    title='Detalhamento do Programa Interlegis',
    xaxis=dict(
        title='Periodo',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='Quantidade',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    )
)

py.iplot(dict(data = [{
    'x': df_final.index,
    'y': df_final[col],
    'name': col
}  for col in df_final.columns[:]], layout = layout), kind='scatter', asFigure=True)

In [7]:
# Kind = tipo de plot; labels = nome dos valores; values = os valores a serem colocados no plot;
df_stats.iplot(kind='pie', labels='TipoServico', values='Total', pull=.1, hole=.1,
         colorscale='blues', textposition='outside', textinfo='value+percent', title='Detalhamento do Programa Interlegis')

In [8]:
# Obs.: Usar sudo pip install XlsxWriter no seu Virtualenv
writer = pd.ExcelWriter('output.xlsx', engine='xlsxwriter')
df_final.loc[:,:].to_excel(writer,'Sheet1')
df_stats.to_excel(writer,'Sheet2')
writer.save()